In [2]:
import pandas as pd
from dfply import *
import numpy as np
from plotnine import *
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import ttest_ind

In [14]:
#RDQ: Different Types of Treatment Outcomes
rdqPre = pd.read_spss('../Data/PrePostMeasures/RDQPre_1.sav') #Pre-treatment
rdqPost = pd.read_spss('../Data/PrePostMeasures/RDQPost_1.sav') #Post-treatment

#Discharge Package: Patient perceived satisfaction & Improvement 
satOld = pd.read_spss('../Data/PHPSatisfaction/EndofTreatmentSurveyData/EOT Verfication_1.sav')#Old data before 2020
satNew = pd.read_spss('../Data/DischargePacket/2022-06-06 dc.sav') #New data after 2020
#We swtiched the platform for data collection in 2020 so we have two separate datasets

#Diagnosis
dx = pd.read_spss('../Data/DemosDx/Diagnosis_1.sav') 

#Demographicsb
demo = pd.read_spss('../Data/DemosDx/Demographics Form_1.sav')

#log
log = pd.read_spss('../Data/DemosDx/Tx Day Consent Log_1.sav')
scided = log[log.SCID_1 == 'yes']

In [82]:
    
#Rename variables to ensure consistency between the old and new datasets of the global rating of improvement
satGlobalOld = satOld.loc[:,['PHP_ID_1', 'IMPRV_1']].rename(columns = {'PHP_ID_1': 'ID1','IMPRV_1': 'imprv1'})
satGlobalNew = satNew.loc[:,['id1', 'imprv_1']].rename(columns = {'id1': 'ID1','imprv_1': 'imprv1'})

#Merge old and new discharge package (i.e., overall satisfaction/improvement) datasets
imprv = pd.concat([satGlobalOld, satGlobalNew],axis = 0) #Use concact to merge data by columns
imprv = imprv[imprv.ID1 != 0] #Remove invalid IDs
imprv.ID1 = imprv.ID1.astype('object') #Change ID type from float to object

In [23]:
rdqPreFilled = rdqPre.isna().sum(axis = 1) < 60
rdqPostFilled = rdqPost.isna().sum(axis = 1) < 60



In [62]:
dx

,ID1,INITIALS_1,DOB_1,none_1,etohab_1,etohdp_1,amphab_1,amphdp_1,mjab_1,mjdep_1,...,addhpr_1,addnos_1,adltadd_1,condct_1,other_1_1,other_1_dx_1,other_2_1,other_2_dx_1,other_3_1,other_3_dx_1
0,1.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
1,2.0,,NaN,0.0,Not present,Curr,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Curr,Bereavement,Curr,Cognitive Disorder NOS,Not present,
2,3.0,,NaN,0.0,Not present,Curr,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
3,4.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
4,5.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7297,7298.0,,NaN,0.0,Curr,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,0,Not present,0,Not present,
7298,7299.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,0,Not present,0,Not present,
7299,7300.0,,NaN,0.0,PR,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,0,Not present,0,Not present,
7300,7301.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,0,Not present,0,Not present,


In [103]:
dx_mdd = dx.loc[dx['mddrnp_1'].str.contains('Curr', na=True) | dx['mddrp_1'].str.contains('Curr', na=True) |
           dx['mddsnp_1'].str.contains('Curr', na=True) | dx['mddsp_1'].str.contains('Curr', na=True) ,:]

dx_mdd

,ID1,INITIALS_1,DOB_1,none_1,etohab_1,etohdp_1,amphab_1,amphdp_1,mjab_1,mjdep_1,...,addhpr_1,addnos_1,adltadd_1,condct_1,other_1_1,other_1_dx_1,other_2_1,other_2_dx_1,other_3_1,other_3_dx_1
1,2.0,,NaN,0.0,Not present,Curr,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Curr,Bereavement,Curr,Cognitive Disorder NOS,Not present,
2,3.0,,NaN,0.0,Not present,Curr,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
5,6.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
6,7.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
7,8.0,,NaN,0.0,Not present,Past,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7294,7295.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,0,Not present,0,Not present,
7297,7298.0,,NaN,0.0,Curr,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,0,Not present,0,Not present,
7298,7299.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,0,Not present,0,Not present,
7300,7301.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,0,Not present,0,Not present,


In [117]:
dx_nonmdd = dx.loc[:,~(dx.columns.str.startswith('mdd')|dx.columns.str.startswith('dysth'))]
dx_nonmdd = dx_nonmdd[-dx_nonmdd.stack(dropna=False).str.contains('Curr').unstack().any(axis=1)]
dx_nonmdd


,ID1,INITIALS_1,DOB_1,none_1,etohab_1,etohdp_1,amphab_1,amphdp_1,mjab_1,mjdep_1,...,addhpr_1,addnos_1,adltadd_1,condct_1,other_1_1,other_1_dx_1,other_2_1,other_2_dx_1,other_3_1,other_3_dx_1
21,22.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
74,75.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
87,88.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
88,89.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
97,98.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,7263.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Past,NaN,,NaN,
7263,7264.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Past,NaN,,NaN,
7264,7265.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Past,NaN,,NaN,
7265,7266.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Past,NaN,,NaN,


In [118]:
mddonly = pd.merge(dx_mdd, dx_nonmdd, how = 'inner', on = 'ID1') 
mddonly = mddonly[mddonly.isnull().sum(axis = 1) < 246]
mddonly.to_csv('../Results/mddonly.csv')
mddonly

,ID1,INITIALS_1_x,DOB_1_x,none_1_x,etohab_1_x,etohdp_1_x,amphab_1_x,amphdp_1_x,mjab_1_x,mjdep_1_x,...,addhpr_1_y,addnos_1_y,adltadd_1_y,condct_1_y,other_1_1_y,other_1_dx_1_y,other_2_1_y,other_2_dx_1_y,other_3_1_y,other_3_dx_1_y
0,22.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
1,75.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
2,88.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
3,89.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
4,98.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,Not present,Not present,Not present,Not present,Not present,Past,Not present,,Not present,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,7263.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Past,NaN,,NaN,
383,7264.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Past,NaN,,NaN,
384,7265.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Past,NaN,,NaN,
385,7266.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Past,NaN,,NaN,


In [109]:
rdqPre_mddonly = pd.merge(rdqPre[rdqPre.isnull().sum(axis = 1) < 60], mddonly, on = 'ID1', how = 'inner') #227

In [110]:
rdqPost_mddonly = pd.merge(rdqPost[rdqPost.isnull().sum(axis = 1) < 60], mddonly, on = 'ID1', how = 'inner') #227

In [111]:
imprv_mddonly = pd.merge(imprv, mddonly, on = 'ID1', how = 'inner') #135

In [112]:
print(rdqPre_mddonly.shape[0],
rdqPost_mddonly.shape[0],
imprv_mddonly.shape[0])

320 210 178


In [113]:
pd.merge(mddonly, scided, on = 'ID1', how = 'inner')

,ID1,INITIALS_1_x,DOB_1_x,none_1_x,etohab_1_x,etohdp_1_x,amphab_1_x,amphdp_1_x,mjab_1_x,mjdep_1_x,...,Prev_AdmitID_1,SCID_1,SCID_date_1,GAF_1,Days_cancel_1,Days_noshow_1,Referral_1,Employment_duration_1,ACT_track_1,Coping_track_1
0,98.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,NaN,yes,2014-06-20,55.0,0.0,0.0,PCP,NaN,NaN,NaN
1,127.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,NaN,yes,2014-07-03,45.0,3.0,3.0,Therapist,48.0,NaN,NaN
2,215.0,,NaN,0.0,Not present,Past,Not present,Not present,Not present,Not present,...,NaN,yes,2014-08-20,55.0,0.0,0.0,Therapist,NaN,NaN,NaN
3,246.0,,NaN,0.0,Not present,Not present,Not present,Not present,Not present,Not present,...,NaN,yes,2014-08-29,45.0,0.0,0.0,Therapist,36.0,NaN,NaN
4,594.0,,NaN,0.0,Not present,Past,Not present,Not present,Not present,Not present,...,NaN,yes,2015-02-05,40.0,0.0,0.0,Therapist,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,7259.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,7260.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,7261.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,7262.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
all_mddonly = pd.merge(rdqPre_mddonly, rdqPost_mddonly, on = 'ID1', how = 'inner')
all_mddonly = pd.merge(all_mddonly , imprv_mddonly, on = 'ID1', how = 'inner')
all_mddonly = pd.merge(all_mddonly , scided, on = 'ID1', how = 'inner')
all_mddonly.shape[0]

59